In [1]:
%matplotlib notebook
from __future__ import division

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from matplotlib.pyplot import plot, ylabel, xlabel, yscale, xscale, legend, subplots
from theano import function
import numpy as np
import gzip
import cPickle
from scipy.optimize import minimize
from climin.util import optimizer
from itertools import repeat, cycle, islice, izip
inf = float("inf")

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [3]:
from breze.learn.data import one_hot
from breze.learn.base import cast_array_to_local_type
from schlichtanders.myfunctools import compose
from schlichtanders.myoptimizers import batch
from schlichtanders.mygenerators import eatN, chunk, every
from schlichtanders.myplot import add_val, add_point

In [4]:
from theano_models import (as_tensor_variable, total_size, clone, clone_all,
                           Merge, FlatKey, Reparameterize, squareplus, squareplus_inv,
                           InvertibleModel,
                           inputting_references, outputting_references)
from theano_models.visualization import d3viz
from IPython.display import IFrame
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
import theano_models.postmaps as post
from theano_models.composing import normalizing_flow, variational_bayes

In [5]:
from theano_models.util.theano_helpers import get_dependencies, sort_dependent_last

In [6]:
from theano.printing import debugprint

In [7]:
inputting_references.update(['to_be_randomized'])
inputting_references

{'flat',
 'inputs',
 'inverse_inputs',
 'n_data',
 'parameters',
 'parameters_positive',
 'parameters_pvalues',
 'to_be_randomized'}

In [8]:
outputting_references

{'inverse_outputs', 'kl_prior', 'logP', 'loglikelihood', 'norm_det', 'outputs'}

# Data

In [9]:
datafile = '../data/mnist.pkl.gz'
# Load data.        a                                                                                           

with gzip.open(datafile,'rb') as f:                                                                        
    train_set, val_set, test_set = cPickle.load(f)                                                       

X, Z = train_set                                                                                               
VX, VZ = val_set
TX, TZ = test_set

Z = one_hot(Z, 10)
VZ = one_hot(VZ, 10)
TZ = one_hot(TZ, 10)

image_dims = 28, 28

X, Z, VX, VZ, TX, TZ = [cast_array_to_local_type(i) for i in (X, Z, VX,VZ, TX, TZ)]
map(np.shape, [X, Z, VX, VZ, TX, TZ])

[(50000, 784),
 (50000, 10),
 (10000, 784),
 (10000, 10),
 (10000, 784),
 (10000, 10)]

# Model

## data modelling

In [10]:
predictor = dm.Mlp(output_size=10, output_transfer="softmax", hidden_sizes=[200]*2, hidden_transfers=["rectifier"]*2)
# post.flatten_parameters(predictor)
predictor

Mlp { 'inputs': [AffineNonlinear.inputs.0],
  'outputs': AffineNonlinear3.outputs,
  'parameters': [weights, bias, weights, bias, weights, bias]}

In [11]:
target_distribution = pm.Categorical(predictor)
target_distribution

Categorical { 'inputs': [],
  'logP': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'outputs': Categorical.outputs,
  'parameters_pvalues': [AffineNonlinear3.outputs]}

In [12]:
target_distribution

Categorical { 'inputs': [],
  'logP': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'outputs': Categorical.outputs,
  'parameters_pvalues': [AffineNonlinear3.outputs]}

In [13]:
targets = Merge(target_distribution, predictor, FlatKey(predictor, flat_key="to_be_randomized"))
targets

Merge { 'inputs': [AffineNonlinear.inputs.0],
  'logP': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'outputs': Categorical.outputs,
  'parameters': [],
  'parameters_pvalues': [],
  'to_be_randomized': "weights_copy:bias_copy:weights_copy:bias_copy:weights_copy:bias_copy"}

## parameter modelling

In [14]:
params_base = pm.DiagGauss(output_size=total_size(targets['to_be_randomized']))  # if you want to use size directly, CAUTION, you need to copy before!
# params_base.map('parameters_positive', reparameterize_map(squareplus, squareplus_inv), 'parameters')
params_base

DiagGauss { 'inputs': [],
  'logP': <FunctionWrapper at 0x7fdb8dfb6d70 for function at 0x7fdb8e091d70>,
  'outputs': DiagGaussianNoise.outputs,
  'parameters': [mean],
  'parameters_positive': [var]}

In [15]:
normflows = [dm.PlanarTransform() for _ in range(2)]
normflows

[PlanarTransform { 'inputs': [z],
   'norm_det': PlanarTransform.norm_det,
   'outputs': PlanarTransform.outputs,
   'parameters': [b, w, _u]}, PlanarTransform2 { 'inputs': [z],
   'norm_det': PlanarTransform2.norm_det,
   'outputs': PlanarTransform2.outputs,
   'parameters': [b, w, _u]}]

In [16]:
params = params_base
for transform in normflows:
    params = normalizing_flow(transform, params)  # returns transform, however with adapted logP    

params

normalized_flow2 { 'inputs': [],
  'logP': <FunctionWrapper at 0x7fdb8defd1a0 for function at 0x7fdb8deab578>,
  'norm_det': PlanarTransform2.norm_det,
  'outputs': PlanarTransform2.outputs,
  'parameters': [b, w, _u, b, w, _u, mean],
  'parameters_positive': [var]}

## bayes

In [17]:
prior = pm.DiagGauss(targets['to_be_randomized'].size)
prior.mean.name = "mean_prior"
prior

DiagGauss2 { 'inputs': [],
  'logP': <FunctionWrapper at 0x7fdb8defd210 for function at 0x7fdb8deab320>,
  'outputs': DiagGaussianNoise2.outputs,
  'parameters': [mean_prior],
  'parameters_positive': [var]}

In [18]:
model = variational_bayes(targets, 'to_be_randomized', params, priors=prior)
model

variational_lower_bound { 'inputs': [AffineNonlinear.inputs.0],
  'kl_prior': kl_prior,
  'logP': <FunctionWrapper at 0x7fdb8dee9360 for function at 0x7fdb8deb40c8>,
  'loglikelihood': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'n_data': n_data,
  'norm_det': PlanarTransform2.norm_det,
  'outputs': Categorical.outputs,
  'parameters': [b, w, _u, b, w, _u, mean, mean_prior],
  'parameters_positive': [var, var],
  'parameters_pvalues': []}

In [19]:
model = Merge(model, Reparameterize(model['parameters_positive'], squareplus, squareplus_inv))
model

Merge2 { 'inputs': [AffineNonlinear.inputs.0],
  'kl_prior': kl_prior,
  'logP': <FunctionWrapper at 0x7fdb8dee9360 for function at 0x7fdb8deb40c8>,
  'loglikelihood': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'n_data': n_data,
  'norm_det': PlanarTransform2.norm_det,
  'outputs': Categorical.outputs,
  'parameters': [ b,
                  w,
                  _u,
                  b,
                  w,
                  _u,
                  mean,
                  mean_prior,
                  var_squareplus,
                  var_squareplus],
  'parameters_positive': [],
  'parameters_pvalues': []}

In [20]:
model = Merge(model, FlatKey(model, initial_inputs=[X[0]]))
model

Merge3 { 'flat': "b:w:_u:b:w:_u:mean:mean_prior:var_squareplus:var_squareplus",
  'inputs': [AffineNonlinear.inputs.0],
  'kl_prior': kl_prior,
  'logP': <FunctionWrapper at 0x7fdb8dee9360 for function at 0x7fdb8deb40c8>,
  'loglikelihood': <FunctionWrapper at 0x7fdb8e068c90 for function at 0x7fdb8e0ff848>,
  'n_data': n_data,
  'norm_det': PlanarTransform2.norm_det,
  'outputs': Categorical.outputs,
  'parameters': [],
  'parameters_positive': [],
  'parameters_pvalues': []}

# Optimizer

In [21]:
InvertibleModel.reduce_all_identities()

In [22]:
postmap = compose(post.flat_numericalize_postmap, post.variational_postmap)
postmap_kwargs = {
    'wrapper': batch,
    'initial_inputs': [X[0]],
    'adapt_init_params': lambda ps: ps + np.random.normal(size=ps.size) * 0.01
}
optimizer_kwargs = postmap(model, **postmap_kwargs)
climin_kwargs = post.climin_postmap(optimizer_kwargs)
climin_kwargs

{'f': <function schlichtanders.myoptimizers.f_batch>,
 'fprime': <function schlichtanders.myoptimizers.f_batch>,
 'wrt': array([ 0.00437581,  1.0095916 ,  0.99145387, ...,  1.00682627,
         0.99813531,  1.00202845])}

Climin wants an iterator of (args, kwarsg) as keyword argument "args" (to be passed to the loss function). Concretley, we use an infinite iterator over minibatches.

In [23]:
batch_size = 20
climin_args = izip(izip(chunk(batch_size, cycle(Z)), chunk(batch_size, cycle(X))), repeat({}))

In [24]:
opt = optimizer(
    identifier = "adadelta",
    args=climin_args, # repeat(((Z,X),{})),
    **climin_kwargs
)

/home/stephan/GitProjects/breze/src/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.adadelta.Adadelta'>
  % (i, klass))


# Visualizing model

In [25]:
# f = theano.function([model['flat']] + model['inputs'], model['outputs'], mode="FAST_COMPILE")
f = [model['flat']] + model['inputs'], model['outputs']
d3viz(f, 'tmp/model.html') #, [targets, target_distribution, predictor, params] + predictor.layers + normflows + [params_base] + Helper.all_helpers[::-1])
IFrame('tmp/model.html', width=700, height=500)

# Visualizing Loss

In [26]:
# f = theano.function([model['flat']] + model['inputs'], optimizer_kwargs['loss'], mode="FAST_COMPILE", profile=True)
f = [model['flat']] + model['inputs'], optimizer_kwargs['loss']
d3viz(f, 'tmp/loss.html')
IFrame('tmp/loss.html', width=700, height=500)

# Visualized Fit

In [ ]:
line_train, = plot([], [], 'go-', label="average training loss")
line_curr_val, = plot([],[], 'bo:', label="avrg current validation loss")
line_best_val, = plot([], [], 'ko-', label="avrg best validation loss")
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
yscale('log')
ylabel("validation loss")
xlabel("#iteration")
legend(loc='lower left', fancybox=True, framealpha=0.5)

In [ ]:
best_val_loss = inf
best_wrt = None
val_size = batch_size

n_whole_data = X.shape[0] // batch_size

for info in every(n_whole_data, opt):
    # collect and visualize validation loss for choosing the best model
    val_loss = optimizer_kwargs['num_loss'](opt.wrt, VZ[:val_size], VX[:val_size])/val_size
    if val_loss < best_val_loss:
        best_wrt = opt.wrt
        best_val_loss = val_loss
        add_point(line_best_val, info['n_iter'], val_loss)
    add_point(line_curr_val, info['n_iter'], val_loss)
    
    # visualize training loss for comparison:
    try:
        training_loss = info['loss'] / len(Z)  # TODO normalization needed?
    except KeyError:
        training_loss = optimizer_kwargs['num_loss'](opt.wrt, Z, X)/len(Z)
    add_point(line_train, info['n_iter'], training_loss)

# Performance

TODO: average over predictions

In [ ]:
print best_val_loss
mlp['parameters_flat'] = best_wrt

predict = mlp.function()
predict(X[0]), Z[0]

In [ ]:
PX = np.apply_along_axis(predict, 1, X)
PVX = np.apply_along_axis(predict, 1, VX)
PTX = np.apply_along_axis(predict, 1, TX)

In [ ]:
print 'incorrect samples train/val/test:  %i/%i/%i' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).sum(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).sum(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).sum())

print 'error rate train/val/test:  %g/%g/%g' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).mean(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).mean(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).mean())